In [74]:
# load libraries
%matplotlib inline
import pandas as pd
import numpy as np
import time
import os
import statistics as stats
import math
from scipy import stats as tests
import researchpy as rp

In [75]:
#see how many rPPG files there are
files = [f for f in os.listdir('rppg_csvs') if not f.startswith('.')]
num_rppgs = len(files)

#load in all of the rPPG data into a single list of dataframes
rppg_data = []
for i in range(num_rppgs):
    userID = os.path.splitext(files[i])[0][:5]
    user_data = pd.read_csv("rppg_csvs/"+userID+".mp4-heart-rate.csv", header=0)
    rppg_data.append((userID,user_data))

In [76]:
#see how many facial affect files there are
files = [f for f in os.listdir('subject_csvs_clean') if not f.startswith('.')]
num_subjs = len(files)

#load in all of the facial affect data into a single list of dataframes
affect_data = []
for i in range(num_subjs):
    userID = os.path.splitext(files[i])[0]
    user_data = pd.read_csv("subject_csvs_clean/"+userID+".csv", header=0)
    affect_data.append((userID,user_data))

In [77]:
#Read in the timestamp and demographic data from the database
df_timestamps = pd.read_csv("timestamps.csv", header=0)
df_timestamps.head()

,userID,prolificID,name,date,vid_consent,starttime,age,gender,wherelive,mood,...,vote,likeJoe,likeTrump,abortion,guncontrol,healthcare,policing,studyabout,finishtime,comment
0,2clJY,5c19a630257db80001a2b3a8,paul benevich,11/24/20,canUseVid,10:11:26 PM,65,Male,Yes,4,...,Trump,1,5,ProLife,AntiGunControl,AntiHealthcare,ProPolice,politics,10:31:41 PM,NaN
1,ifh7Z,5e8790801eb5090afd11e178,Caitlin Sisk,11/24/20,canUseVid,10:18:00 PM,26,Female,Yes,2,...,Biden,3,2,ProLife,ProGunControl,AntiHealthcare,ProPolice,How emotional reactions to politicians voicing...,10:38:37 PM,NaN
2,1XVI6,5ed70903f3b2f8195f919e6f,Charles Edward Geyer,11/24/20,canUseVid,10:21:06 PM,35,Male,Yes,4,...,Trump,2,4,ProLife,AntiGunControl,AntiHealthcare,ProPolice,What your reactions are to policies forwarded ...,10:42:36 PM,NaN
3,PPckh,5f4926497bf47b4392f9b163,Sara Owen,25-Nov-20,canUseVid,4:15:02 PM,21,Other,Yes,4,...,Biden,3,1,ProChoice,AntiGunControl,ProHealthcare,AntiPolice,How people&#39;s reactions change when they ar...,4:33:48 PM,NaN
4,yFcTy,5faef59ae8f4f50f9ab78386,Nathan Yousman,11/25/20,canUseVid,4:56:21 PM,19,Male,Yes,3,...,Biden,3,1,ProChoice,ProGunControl,ProHealthcare,AntiPolice,Showing people videos of the two major candida...,5:15:40 PM,NaN


In [78]:
# IFF we get a negative result, we need to set noon and midnight equal to 0:00
# while avoiding doing that if it tips form 11:00 to 12:00
def clocks_are_hard(time):
    if time < 0:
        return time + (3600*12)
    else:
        return time

In [79]:
#Get all of the video timestimes and put them in the same order
def retreive_vid_times(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    times = df_timestamps[df_timestamps["userID"] == userID]
    vid_order = times.at[times.index[0],'vidOrder'].split(',')
    time_array = times.at[times.index[0],"starttime"].split()[0].split(':')
    start_time = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])
    
    vid_times = [[0,0],[0,0],[0,0],[0,0]]
    for i in range(len(vid_order)):
        if vid_order[i] == '0':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[0][0] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[0][1] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
        elif vid_order[i] == '1':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[1][0] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[1][1] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
        elif vid_order[i] == '2':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[2][0] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[2][1] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
        elif vid_order[i] == '3':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[3][0] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[3][1] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)

    return vid_times

In [80]:
print(retreive_vid_times(6))

[[1196.0, 1462.0], [1518.0, 1714.0], [1751.0, 1978.0], [972.0, 1153.0]]


In [81]:
def retreive_affiliation(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    responses = df_timestamps[df_timestamps["userID"] == userID]
    party = responses.at[responses.index[0],'party']
    ideology = responses.at[responses.index[0],'ideology']
    candidate = responses.at[responses.index[0],'vote']
    like_joe = responses.at[responses.index[0],'likeJoe']
    like_trump = responses.at[responses.index[0],'likeTrump']

    return [userID, party,ideology,candidate,like_joe,like_trump]

In [82]:
print(retreive_affiliation(6))

['lCWiX', 'Democrat', 'Left', 'Biden', 5, 1]


In [83]:
#Create a master list of which timestamps correspond to which videos for each user
compiled_vid_times = []
for i in range(num_subjs):
    compiled_vid_times.append(retreive_vid_times(i))
compiled_vid_times

[[[357.0, 562.0], [849.0, 1025.0], [612.0, 795.0], [107.0, 303.0]],
 [[803.0, 1082.0], [305.0, 490.0], [551.0, 764.0], [66.0, 262.0]],
 [[833.0, 1053.0], [369.0, 549.0], [594.0, 779.0], [141.0, 326.0]],
 [[748.0, 948.0], [541.0, 705.0], [319.0, 500.0], [112.0, 279.0]],
 [[855.0, 1070.0], [382.0, 560.0], [601.0, 802.0], [149.0, 338.0]],
 [[454.0, 674.0], [964.0, 1167.0], [222.0, 403.0], [721.0, 917.0]],
 [[1196.0, 1462.0], [1518.0, 1714.0], [1751.0, 1978.0], [972.0, 1153.0]],
 [[935.0, 1133.0], [259.0, 448.0], [709.0, 897.0], [499.0, 668.0]],
 [[909.0, 1128.0], [71.0, 284.0], [353.0, 580.0], [642.0, 853.0]],
 [[982.0, 1056.0], [118.0, 345.0], [711.0, 918.0], [434.0, 649.0]],
 [[632.0, 896.0], [946.0, 1156.0], [359.0, 576.0], [79.0, 282.0]],
 [[314.0, 521.0], [559.0, 743.0], [797.0, 988.0], [77.0, 265.0]],
 [[443.0, 679.0], [974.0, 1174.0], [194.0, 394.0], [730.0, 930.0]],
 [[372.0, 602.0], [116.0, 311.0], [936.0, 1131.0], [678.0, 869.0]],
 [[291.0, 493.0], [532.0, 699.0], [741.0, 924.0]

In [84]:
sentiment_df = pd.read_csv("text_sentiment.csv", header=0)
sentiment_df.head()

,userID,vid1_pol,vid1_subj,vid2_pol,vid2_subj,vid3_pol,vid3_subj,vid4_pol,vid4_subj
0,1XVI6,0.1125,0.5984,-0.0417,0.5542,0.1789,0.4378,-0.0654,0.6196
1,5yMwL,-0.0833,0.3333,0.2000,0.5000,-0.8000,0.8000,-0.5750,1.0000
2,GFPux,0.2469,0.4031,0.0750,0.3417,-0.0357,0.5929,0.0667,0.5264
3,mOhA6,0.4700,0.5800,0.4333,0.4667,0.3917,0.5083,-0.0500,0.5917
4,mTe10,-0.1500,0.6444,0.2600,0.6500,0.7000,0.6000,0.0000,0.0000


In [85]:
#compute summary statistics for each video for each participant
def compute_summary_stats(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    vid_times = compiled_vid_times[index]
    
    for r in rppg_data:
        if r[0] == userID:
            rppg_df = r[1]
            break
            
    disgust = []
    joy = []
    engagement = []
    valence = []
    rppg = []
    
    print(userID)  #progress tracker

    #compute the average affect in each facial affect category during the relevant times
    for i in range(4):
        time1 = vid_times[i][0]
        time2 = vid_times[i][1]
        rows = affect_df['time_stamp'].between(time1, time2)
        temp1 = []
        temp2 = []
        temp3 = []
        temp4 = []
        for j in rows.index:
            if rows[j]:
                temp1.append(affect_df.at[j,'disgust'])
                temp2.append(affect_df.at[j,'joy'])
                temp3.append(affect_df.at[j,'engagement'])
                temp4.append(affect_df.at[j,'valence'])
        disgust.append(stats.mean(temp1))
        joy.append(stats.mean(temp2))
        engagement.append(stats.mean(temp3))
        valence.append(stats.mean(temp4))
        
    #compute the average rPPG during the relevant times
    end_time = max(affect_df["time_stamp"])
    secs_per_bin = end_time / 240
    for i in range(4):
        #rPPG data is always split into 240 equal time bins
        time1 = vid_times[i][0]/secs_per_bin
        time2 = vid_times[i][1]/secs_per_bin
        rows = rppg_df['timebin'].between(time1, time2)
        temp1 = []
        for j in rows.index:
            if rows[j]:
                temp1.append(rppg_df.at[j,'hb-smooth'])
        rppg.append(stats.mean(temp1))

    #pull in the text sentiment polarity and subjectivity.  Values are -1 to 1, so *100 to compare with facial affect.
    sentiment = sentiment_df[sentiment_df["userID"] == userID]
    vid1_pol = sentiment.at[sentiment.index[0],'vid1_pol']*100
    vid2_pol = sentiment.at[sentiment.index[0],'vid2_pol']*100
    vid3_pol = sentiment.at[sentiment.index[0],'vid3_pol']*100
    vid4_pol = sentiment.at[sentiment.index[0],'vid4_pol']*100
    vid1_subj = sentiment.at[sentiment.index[0],'vid1_subj']*100
    vid2_subj = sentiment.at[sentiment.index[0],'vid2_subj']*100
    vid3_subj = sentiment.at[sentiment.index[0],'vid3_subj']*100
    vid4_subj = sentiment.at[sentiment.index[0],'vid4_subj']*100
    polarity_init = [vid1_pol,vid2_pol,vid3_pol,vid4_pol]
    subjectivity_init = [vid1_subj,vid2_subj,vid3_subj,vid4_subj]
    
    #reorder text sentiment based on video order to line up with the rest of the analysis
    times = df_timestamps[df_timestamps["userID"] == userID]
    vid_order = times.at[times.index[0],'vidOrder'].split(',')
    polarity = [0,0,0,0]
    subjectivity = [0,0,0,0]
    for i in range(len(vid_order)):
        if vid_order[i] == '0':
            polarity[0] = polarity_init[i]
            subjectivity[0] = subjectivity_init[i]
        elif vid_order[i] == '1':
            polarity[1] = polarity_init[i]
            subjectivity[1] = subjectivity_init[i]
        elif vid_order[i] == '2':
            polarity[2] = polarity_init[i]
            subjectivity[2] = subjectivity_init[i]
        elif vid_order[i] == '3':
            polarity[3] = polarity_init[i]
            subjectivity[3] = subjectivity_init[i]
        
    #Pull in user affiliation
    aff = retreive_affiliation(index)
    party = aff[1]
    ideology = aff[2]
    candidate = aff[3]
    like_joe = aff[4]
    like_trump = aff[5]
    
    #apply data to output variables based on user affiliation
    #currently we only accept data aligned (Dem-Lib-Biden or Rep-Con-Trump) so other cases are ignored
    if party == "Democrat":
        disgust_aligned = disgust[3]
        disgust_misaligned_proi = disgust[2]
        disgust_misaligned_prop = disgust[1]
        disgust_opposed = disgust[0]
        joy_aligned = joy[3]
        joy_misaligned_proi = joy[2]
        joy_misaligned_prop = joy[1]
        joy_opposed = joy[0]
        engagement_aligned = engagement[3]
        engagement_misaligned_proi = engagement[2]
        engagement_misaligned_prop = engagement[1]
        engagement_opposed = engagement[0]
        valence_aligned = valence[3]
        valence_misaligned_proi = valence[2]
        valence_misaligned_prop = valence[1]
        valence_opposed = valence[0]
        polarity_aligned = polarity[3]
        polarity_misaligned_proi = polarity[2]
        polarity_misaligned_prop = polarity[1]
        polarity_opposed = polarity[0]
        subjectivity_aligned = subjectivity[3]
        subjectivity_misaligned_proi = subjectivity[2]
        subjectivity_misaligned_prop = subjectivity[1]
        subjectivity_opposed = subjectivity[0]
        rppg_aligned = rppg[3]
        rppg_misaligned_proi = rppg[2]
        rppg_misaligned_prop = rppg[1]
        rppg_opposed = rppg[0]
    else:
        disgust_aligned = disgust[0]
        disgust_misaligned_proi = disgust[1]
        disgust_misaligned_prop = disgust[2]
        disgust_opposed = disgust[3]
        joy_aligned = joy[0]
        joy_misaligned_proi = joy[1]
        joy_misaligned_prop = joy[2]
        joy_opposed = joy[3]
        engagement_aligned = engagement[0]
        engagement_misaligned_proi = engagement[1]
        engagement_misaligned_prop = engagement[2]
        engagement_opposed = engagement[3]
        valence_aligned = valence[0]
        valence_misaligned_proi = valence[1]
        valence_misaligned_prop = valence[2]
        valence_opposed = valence[3]
        polarity_aligned = polarity[0]
        polarity_misaligned_proi = polarity[1]
        polarity_misaligned_prop = polarity[2]
        polarity_opposed = polarity[3]
        subjectivity_aligned = subjectivity[0]
        subjectivity_misaligned_proi = subjectivity[1]
        subjectivity_misaligned_prop = subjectivity[2]
        subjectivity_opposed = subjectivity[3]
        rppg_aligned = rppg[0]
        rppg_misaligned_proi = rppg[1]
        rppg_misaligned_prop = rppg[2]
        rppg_opposed = rppg[3]

    return [userID,party,ideology,candidate,like_joe,
              like_trump,disgust_aligned,disgust_misaligned_proi,
              disgust_misaligned_prop,disgust_opposed,joy_aligned,joy_misaligned_proi,
              joy_misaligned_prop,joy_opposed,engagement_aligned,engagement_misaligned_proi,
              engagement_misaligned_prop,engagement_opposed,valence_aligned,
              valence_misaligned_proi,valence_misaligned_prop,valence_opposed,
              polarity_aligned,polarity_misaligned_proi,polarity_misaligned_prop,
              polarity_opposed,subjectivity_aligned,subjectivity_misaligned_proi,
              subjectivity_misaligned_prop,subjectivity_opposed,rppg_aligned,
              rppg_misaligned_proi,rppg_misaligned_prop,rppg_opposed]

In [86]:
print(compute_summary_stats(9))

mTe10
['mTe10', 'Democrat', 'Left', 'Biden', 5, 1, 0.49363620000457914, 0.5293182388624262, 0.46429756486199103, 0.6651844879434513, 0.6098790342662332, 0.0018139395116161575, 0.0027608550985296944, 0.002294039240448924, 14.434702684472745, 11.87562420677953, 11.648467810037422, 22.95363114693947, 3.436418944400033, -3.861992109636242, 1.3668263880371125, -6.278409160589113, 26.0, 70.0, -15.0, 0.0, 65.0, 60.0, 64.44, 0.0, 10.573623913043479, 15.422062222222223, 4.943149673469388, -2.26955]


In [87]:
#create the summary stats output dataframe

temp_data = [['userID','party','ideology','candidate','like_joe',
              'like_trump','disgust_aligned','disgust_misaligned_proi',
              'disgust_misaligned_prop','disgust_opposed','joy_aligned','joy_misaligned_proi',
              'joy_misaligned_prop','joy_opposed','engagement_aligned','engagement_misaligned_proi',
              'engagement_misaligned_prop','engagement_opposed','valence_aligned',
              'valence_misaligned_proi','valence_misaligned_prop','valence_opposed',
              'polarity_aligned','polarity_misaligned_proi','polarity_misaligned_prop',
              'polarity_opposed','subjectivity_aligned','subjectivity_misaligned_proi',
              'subjectivity_misaligned_prop','subjectivity_opposed','rppg_aligned',
              'rppg_misaligned_proi','rppg_misaligned_prop','rppg_opposed']]
output_df = pd.DataFrame(temp_data, columns = ['userID','party','ideology','candidate','like_joe',
              'like_trump','disgust_aligned','disgust_misaligned_proi',
              'disgust_misaligned_prop','disgust_opposed','joy_aligned','joy_misaligned_proi',
              'joy_misaligned_prop','joy_opposed','engagement_aligned','engagement_misaligned_proi',
              'engagement_misaligned_prop','engagement_opposed','valence_aligned',
              'valence_misaligned_proi','valence_misaligned_prop','valence_opposed',
              'polarity_aligned','polarity_misaligned_proi','polarity_misaligned_prop',
              'polarity_opposed','subjectivity_aligned','subjectivity_misaligned_proi',
              'subjectivity_misaligned_prop','subjectivity_opposed','rppg_aligned',
              'rppg_misaligned_proi','rppg_misaligned_prop','rppg_opposed'])

k = 0
for i in range(num_subjs):
    foo = [compute_summary_stats(i)]
    temp_df = pd.DataFrame(foo, columns = ['userID','party','ideology','candidate','like_joe',
              'like_trump','disgust_aligned','disgust_misaligned_proi',
              'disgust_misaligned_prop','disgust_opposed','joy_aligned','joy_misaligned_proi',
              'joy_misaligned_prop','joy_opposed','engagement_aligned','engagement_misaligned_proi',
              'engagement_misaligned_prop','engagement_opposed','valence_aligned',
              'valence_misaligned_proi','valence_misaligned_prop','valence_opposed',
              'polarity_aligned','polarity_misaligned_proi','polarity_misaligned_prop',
              'polarity_opposed','subjectivity_aligned','subjectivity_misaligned_proi',
              'subjectivity_misaligned_prop','subjectivity_opposed','rppg_aligned',
              'rppg_misaligned_proi','rppg_misaligned_prop','rppg_opposed'])
    output_df.loc[k] = temp_df.loc[0]
    k += 1


j7WOU
GFPux
mOhA6
Ur76R
2clJY
VRQJ7
lCWiX
5yMwL
6aS64
mTe10
1XVI6
yFcTy
zFHlS
WrJ4w
Ssd0G
qWg6e
Crzgd
LxPDL
LAC2O
PPckh
yavQR
20EGr
RWgXF
dSS3V


In [88]:
output_df.to_csv(r'initial_results4_clean.csv', index = False)

In [24]:
### T TEST SECTION ###

In [89]:
#t-test syntax check
tests.ttest_rel(output_df["disgust_aligned"],output_df["disgust_misaligned_proi"])

Ttest_relResult(statistic=-1.3410345409599866, pvalue=0.1930035197675945)

In [90]:
#Prepare the t-test output dataframe
temp_data = [['comparison_var','t-statistic','p-value']]
ttest_df = pd.DataFrame(temp_data, columns = ['comparison_var','t-statistic','p-value'])


ttest = tests.ttest_rel(output_df["disgust_aligned"],output_df["disgust_misaligned_proi"])
ttest_df.loc[0] = pd.DataFrame([['disgust_misaligned_proi',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["disgust_aligned"],output_df["disgust_misaligned_prop"])
ttest_df.loc[1] = pd.DataFrame([['disgust_misaligned_prop',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["disgust_aligned"],output_df["disgust_opposed"])
ttest_df.loc[2] = pd.DataFrame([['disgust_opposed',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]

ttest = tests.ttest_rel(output_df["joy_aligned"],output_df["joy_misaligned_proi"])
ttest_df.loc[3] = pd.DataFrame([['joy_misaligned_proi',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["joy_aligned"],output_df["joy_misaligned_prop"])
ttest_df.loc[4] = pd.DataFrame([['joy_misaligned_prop',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["joy_aligned"],output_df["joy_opposed"])
ttest_df.loc[5] = pd.DataFrame([['joy_opposed',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]

ttest = tests.ttest_rel(output_df["engagement_aligned"],output_df["engagement_misaligned_proi"])
ttest_df.loc[6] = pd.DataFrame([['engagement_misaligned_proi',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["engagement_aligned"],output_df["engagement_misaligned_prop"])
ttest_df.loc[7] = pd.DataFrame([['engagement_misaligned_prop',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["engagement_aligned"],output_df["engagement_opposed"])
ttest_df.loc[8] = pd.DataFrame([['engagement_opposed',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]

ttest = tests.ttest_rel(output_df["valence_aligned"],output_df["valence_misaligned_proi"])
ttest_df.loc[9] = pd.DataFrame([['valence_misaligned_proi',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["valence_aligned"],output_df["valence_misaligned_prop"])
ttest_df.loc[10] = pd.DataFrame([['valence_misaligned_prop',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["valence_aligned"],output_df["valence_opposed"])
ttest_df.loc[11] = pd.DataFrame([['valence_opposed',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]

ttest = tests.ttest_rel(output_df["polarity_aligned"],output_df["polarity_misaligned_proi"])
ttest_df.loc[12] = pd.DataFrame([['polarity_misaligned_proi',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["polarity_aligned"],output_df["polarity_misaligned_prop"])
ttest_df.loc[13] = pd.DataFrame([['polarity_misaligned_prop',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["polarity_aligned"],output_df["polarity_opposed"])
ttest_df.loc[14] = pd.DataFrame([['polarity_opposed',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]

ttest = tests.ttest_rel(output_df["subjectivity_aligned"],output_df["subjectivity_misaligned_proi"])
ttest_df.loc[15] = pd.DataFrame([['subjectivity_misaligned_proi',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["subjectivity_aligned"],output_df["subjectivity_misaligned_prop"])
ttest_df.loc[16] = pd.DataFrame([['subjectivity_misaligned_prop',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["subjectivity_aligned"],output_df["subjectivity_opposed"])
ttest_df.loc[17] = pd.DataFrame([['subjectivity_opposed',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]

ttest = tests.ttest_rel(output_df["rppg_aligned"],output_df["rppg_misaligned_proi"])
ttest_df.loc[18] = pd.DataFrame([['rppg_misaligned_proi',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["rppg_aligned"],output_df["rppg_misaligned_prop"])
ttest_df.loc[19] = pd.DataFrame([['rppg_misaligned_prop',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]
ttest = tests.ttest_rel(output_df["rppg_aligned"],output_df["rppg_opposed"])
ttest_df.loc[20] = pd.DataFrame([['rppg_opposed',ttest[0],ttest[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]

ttest_df.head()

,comparison_var,t-statistic,p-value
0,disgust_misaligned_proi,-1.34103,0.193004
1,disgust_misaligned_prop,-1.88502,0.0721238
2,disgust_opposed,-2.08027,0.0488297
3,joy_misaligned_proi,-1.07711,0.292598
4,joy_misaligned_prop,1.12243,0.273261


In [61]:
ttest_df.to_csv(r'ttest_results_clean.csv', index = False)

In [62]:
### ANOVA SECTION ###

In [91]:
def normalize(df, colname):
    #mean = stats.mean(df[colname].values)
    stdev = stats.stdev(df[colname].values)
    df[colname] = (df[colname])/stdev

In [92]:
output_df.head()

,userID,party,ideology,candidate,like_joe,like_trump,disgust_aligned,disgust_misaligned_proi,disgust_misaligned_prop,disgust_opposed,...,polarity_misaligned_prop,polarity_opposed,subjectivity_aligned,subjectivity_misaligned_proi,subjectivity_misaligned_prop,subjectivity_opposed,rppg_aligned,rppg_misaligned_proi,rppg_misaligned_prop,rppg_opposed
0,j7WOU,Republican,Right,Trump,3,4,0.863112,1.42027,1.66796,0.572138,...,60,20,0,87.5,90,20,19.2167,16.3738,20.4635,7.86823
1,GFPux,Democrat,Left,Biden,3,1,3.22009,6.47995,6.57469,11.7155,...,7.5,6.67,40.31,59.29,34.17,52.64,20.7437,21.1572,14.984,26.8644
2,mOhA6,Democrat,Left,Biden,4,1,0.426519,0.796976,0.436174,0.478588,...,43.33,-5,58,50.83,46.67,59.17,2.69276,28.3208,17.7348,20.2817
3,Ur76R,Democrat,Left,Biden,2,1,0.574838,0.802812,1.00401,0.727271,...,-100,-70,0,70,100,90,29.3361,28.8474,31.5763,36.8766
4,2clJY,Republican,Right,Trump,1,5,9.84002,16.9178,11.7462,11.0466,...,-37.4,-36,72.34,71.9,85.06,64,10.6943,7.04225,11.6757,9.83546


In [93]:
for i in range(6,len(output_df.columns)):
    normalize(output_df, output_df.columns[i])
output_df.head()

,userID,party,ideology,candidate,like_joe,like_trump,disgust_aligned,disgust_misaligned_proi,disgust_misaligned_prop,disgust_opposed,...,polarity_misaligned_prop,polarity_opposed,subjectivity_aligned,subjectivity_misaligned_proi,subjectivity_misaligned_prop,subjectivity_opposed,rppg_aligned,rppg_misaligned_proi,rppg_misaligned_prop,rppg_opposed
0,j7WOU,Republican,Right,Trump,3,4,0.292428,0.366125,0.42642,0.0864012,...,1.88914,0.599268,0,6.84978,4.96147,0.899885,2.15423,2.20832,2.35852,0.731483
1,GFPux,Democrat,Left,Biden,3,1,1.09099,1.67043,1.68085,1.76921,...,0.236142,0.199856,1.70399,4.64141,1.88371,2.3685,2.32541,2.85345,1.72698,2.49749
2,mOhA6,Democrat,Left,Biden,4,1,0.144508,0.205448,0.11151,0.0722737,...,1.36427,-0.149817,2.45179,3.97913,2.5728,2.66231,0.301864,3.8196,2.04403,1.88552
3,Ur76R,Democrat,Left,Biden,2,1,0.194759,0.206953,0.256679,0.109829,...,-3.14857,-2.09744,0,5.47982,5.51275,4.04948,3.28865,3.89063,3.63932,3.42829
4,2clJY,Republican,Right,Trump,1,5,3.33386,4.36115,3.00298,1.6682,...,-1.17756,-1.07868,3.05797,5.62856,4.68914,2.87963,1.19885,0.949783,1.34569,0.914369


In [94]:
tests.f_oneway(output_df['engagement_aligned'],
               output_df['engagement_misaligned_proi'],
               output_df['engagement_misaligned_prop'],
               output_df['engagement_opposed'])

F_onewayResult(statistic=0.9180582126255713, pvalue=0.43542258077130913)

In [95]:
#Prepare the t-test output dataframe
temp_data = [['comparison_var','t-statistic','p-value']]
anova_df = pd.DataFrame(temp_data, columns = ['comparison_var','t-statistic','p-value'])

for i in range(6,len(output_df.columns),4):
    anova = tests.f_oneway(output_df[output_df.columns[i]],
                   output_df[output_df.columns[i+1]],
                   output_df[output_df.columns[i+2]],
                   output_df[output_df.columns[i+3]])
    anova_df.loc[(i/4)-1] = pd.DataFrame([[output_df.columns[i],anova[0],anova[1]]], columns = ['comparison_var','t-statistic','p-value']).loc[0]

anova_df

,comparison_var,t-statistic,p-value
0.0,comparison_var,t-statistic,p-value
0.5,disgust_aligned,0.190633,0.902537
1.5,joy_aligned,0.317877,0.812417
2.5,engagement_aligned,0.918058,0.435423
3.5,valence_aligned,0.304409,0.822136
4.5,polarity_aligned,3.40854,0.0208322
5.5,subjectivity_aligned,44.4136,7.74437e-18
6.5,rppg_aligned,3.53126,0.0178923


In [68]:
anova_df.to_csv(r'anova_results_clean.csv', index = False)

In [ ]:
### EXTRA UTILITY FUNCTIONS ###

In [45]:
def retreive_pc_times(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    times = df_timestamps[df_timestamps["userID"] == userID]
    pc_order = times.at[times.index[0],'pcOrder'].split(',')
    time_array = times.at[times.index[0],"starttime"].split()[0].split(':')
    start_time = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])
    
    pc_times = [[0,0],[0,0],[0,0],[0,0]]
    for i in range(len(pc_order)-1):
        if pc_order[i] == '0':
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            pc_times[0][0] = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+2))].split()[0].split(':')
            pc_times[0][1] = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time
        elif pc_order[i] == '1':
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            pc_times[1][0] = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+2))].split()[0].split(':')
            pc_times[1][1] = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time
        elif pc_order[i] == '2':
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            pc_times[2][0] = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+2))].split()[0].split(':')
            pc_times[2][1] = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time
        elif pc_order[i] == '3':
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            pc_times[3][0] = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+2))].split()[0].split(':')
            pc_times[3][1] = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time
            
    last_vid_pos = int(pc_order[3])
    time_array = times.at[times.index[0],"pc4time"].split()[0].split(':')
    pc_times[last_vid_pos][0] = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time
    pc_times[last_vid_pos][1] = 10000
        
    return pc_times

In [46]:
retreive_pc_times(0)

[[795.0, 849.0], [1025.0, 10000], [562.0, 612.0], [303.0, 357.0]]

In [47]:
compiled_pc_times = []
for i in range(num_subjs):
    compiled_pc_times.append(retreive_pc_times(i))
compiled_pc_times

[[[795.0, 849.0], [1025.0, 10000], [562.0, 612.0], [303.0, 357.0]],
 [[764.0, 803.0], [262.0, 305.0], [490.0, 551.0], [1082.0, 10000]],
 [[326.0, 369.0], [549.0, 594.0], [779.0, 833.0], [1053.0, 10000]],
 [[279.0, 319.0], [500.0, 541.0], [948.0, 10000], [705.0, 748.0]],
 [[338.0, 382.0], [1070.0, 10000], [802.0, 855.0], [560.0, 601.0]],
 [[403.0, 454.0], [1167.0, 10000], [917.0, 964.0], [674.0, 721.0]],
 [[448.0, 499.0], [897.0, 935.0], [668.0, 709.0], [1133.0, 10000]],
 [[649.0, 711.0], [918.0, 982.0], [345.0, 434.0], [1056.0, 10000]],
 [[896.0, 946.0], [576.0, 632.0], [282.0, 359.0], [1156.0, 10000]],
 [[521.0, 559.0], [265.0, 314.0], [988.0, 10000], [743.0, 797.0]],
 [[394.0, 443.0], [930.0, 974.0], [1174.0, 10000], [679.0, 730.0]],
 [[493.0, 532.0], [699.0, 741.0], [244.0, 291.0], [924.0, 10000]],
 [[761.0, 807.0], [248.0, 300.0], [1004.0, 10000], [493.0, 547.0]],
 [[1145.0, 1250.0], [465.0, 559.0], [1473.0, 10000], [794.0, 893.0]],
 [[1015.0, 1057.0], [342.0, 398.0], [632.0, 695.0

In [48]:
#compute summary statistics for palate cleansers
def compute_pc_stats(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    pc_times = compiled_pc_times[index]
    disgust = []
    joy = []
    engagement = []
    valence = []
    
    print(userID)  #progress tracker

    #compute the average affect in each category during the relevant times
    for i in range(4):
        rows = affect_df['time_stamp'].between(vid_times[i][0], vid_times[i][1])
        temp1 = []
        temp2 = []
        temp3 = []
        temp4 = []
        for j in rows.index:
            if rows[j]:
                temp1.append(affect_df.at[j,'disgust'])
                temp2.append(affect_df.at[j,'joy'])
                temp3.append(affect_df.at[j,'engagement'])
                temp4.append(affect_df.at[j,'valence'])
        disgust.append(stats.mean(temp1))
        joy.append(stats.mean(temp2))
        engagement.append(stats.mean(temp3))
        valence.append(stats.mean(temp4))
    

    aff = retreive_affiliation(index)
    party = aff[1]
    ideology = aff[2]
    candidate = aff[3]
    
    output = aff + disgust + joy + engagement + valence

    return output

In [49]:
compute_pc_stats(0)

j7WOU


['j7WOU',
 'Republican',
 'Right',
 'Trump',
 0.8631123712244636,
 1.420274805476483,
 1.6679581630612828,
 0.5721377643406982,
 0.002488020312331199,
 0.0017109820601491667,
 0.001252411364541375,
 0.15116992575035285,
 4.806360774423608,
 0.8442659713238292,
 5.212710304162034,
 1.4335110825507922,
 -2.1724975797555053,
 -2.3547050839163695,
 -6.558213358469032,
 -0.8982007053106487]

In [51]:
#create the pc stats output dataframe

temp_data = [['userID','party','ideology','candidate','pc1disgust','pc2disgust',
              'pc3disgust','pc4disgust','pc1joy','pc2joy',
              'pc3joy','pc4joy','pc1engagement','pc2engagement',
              'pc3engagement','pc4engagement','pc1valence',
              'pc2valence','pc3valence','pc4valence']]
output_df = pd.DataFrame(temp_data, columns = ['userID','party','ideology','candidate','pc1disgust','pc2disgust',
              'pc3disgust','pc4disgust','pc1joy','pc2joy',
              'pc3joy','pc4joy','pc1engagement','pc2engagement',
              'pc3engagement','pc4engagement','pc1valence',
              'pc2valence','pc3valence','pc4valence'])

k = 0
for i in range(num_subjs):
    foo = [compute_pc_stats(i)]
    temp_df = pd.DataFrame(foo, columns = ['userID','party','ideology','candidate','pc1disgust','pc2disgust',
              'pc3disgust','pc4disgust','pc1joy','pc2joy',
              'pc3joy','pc4joy','pc1engagement','pc2engagement',
              'pc3engagement','pc4engagement','pc1valence',
              'pc2valence','pc3valence','pc4valence'])
    output_df.loc[k] = temp_df.loc[0]
    k += 1


j7WOU
GFPux
mOhA6
Ur76R
2clJY
VRQJ7
5yMwL
mTe10
1XVI6
yFcTy
zFHlS
Ssd0G
PPckh
yavQR
20EGr
RWgXF


In [52]:
output_df.to_csv(r'pc_results.csv', index = False)

In [ ]:
#compute t test?  Other form of determining statistical significance?

In [154]:
def retreive_vid_times_unordered(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    times = df_timestamps[df_timestamps["userID"] == userID]
    vid_order = times.at[times.index[0],'vidOrder'].split(',')
    time_array = times.at[times.index[0],"starttime"].split()[0].split(':')
    start_time = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])
    
    vid_times = []
    for i in range(len(vid_order)):
        time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
        vid_times.append(3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time)
        time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
        vid_times.append(3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time)

    return [userID] + vid_times

In [155]:
compiled_vid_times_unordered = []
for i in range(num_subjs):
    compiled_vid_times_unordered.append(retreive_vid_times_unordered(i))
compiled_vid_times_unordered

[['j7WOU', 107.0, 303.0, 357.0, 562.0, 612.0, 795.0, 849.0, 1025.0],
 ['GFPux', 66.0, 262.0, 305.0, 490.0, 551.0, 764.0, 803.0, 1082.0],
 ['mOhA6', 141.0, 326.0, 369.0, 549.0, 594.0, 779.0, 833.0, 1053.0],
 ['VWKWR', 966.0, 1187.0, 1271.0, 1495.0, 1551.0, 1807.0, 1865.0, 2077.0],
 ['Ur76R', 112.0, 279.0, 319.0, 500.0, 541.0, 705.0, 748.0, 948.0],
 ['2clJY', 149.0, 338.0, 382.0, 560.0, 601.0, 802.0, 855.0, 1070.0],
 ['VRQJ7', 222.0, 403.0, 454.0, 674.0, 721.0, 917.0, 964.0, 1167.0],
 ['5yMwL', 259.0, 448.0, 499.0, 668.0, 709.0, 897.0, 935.0, 1133.0],
 ['mTe10', 118.0, 345.0, 434.0, 649.0, 711.0, 918.0, 982.0, 1056.0],
 ['1XVI6', 79.0, 282.0, 359.0, 576.0, 632.0, 896.0, 946.0, 1156.0],
 ['yFcTy', 77.0, 265.0, 314.0, 521.0, 559.0, 743.0, 797.0, 988.0],
 ['zFHlS', 194.0, 394.0, 443.0, 679.0, 730.0, 930.0, 974.0, 1174.0],
 ['Ssd0G', 79.0, 244.0, 291.0, 493.0, 532.0, 699.0, 741.0, 924.0],
 ['ifh7Z', 157.0, 361.0, 411.0, 584.0, 628.0, 812.0, 862.0, 1078.0],
 ['PPckh', 74.0, 248.0, 300.0, 493.

In [160]:
#create the summary stats output dataframe

temp_data = [['userID','vid1start','vid1end','vid2start','vid2end','vid3start','vid3end','vid4start','vid4end']]
output_df = pd.DataFrame(temp_data, columns = ['userID','vid1start','vid1end','vid2start','vid2end','vid3start','vid3end','vid4start','vid4end'])

k = 0
for i in range(num_subjs):
    stats = [retreive_vid_times_unordered(i)]
    temp_df = pd.DataFrame(stats, columns = ['userID','vid1start','vid1end','vid2start','vid2end','vid3start','vid3end','vid4start','vid4end'])
    output_df.loc[k] = temp_df.loc[0]
    k += 1


In [161]:
output_df.to_csv(r'vid_times.csv', index = False)

In [10]:
def retreive_pc_times_unordered(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    times = df_timestamps[df_timestamps["userID"] == userID]
    pc_order = times.at[times.index[0],'pcOrder'].split(',')
    time_array = times.at[times.index[0],"starttime"].split()[0].split(':')
    start_time = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])
    
    pc_times = []
    for i in range(len(pc_order)-1):
        time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
        pc_times.append(3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time)
        time_array = times.at[times.index[0],"vid{x}time".format(x=(i+2))].split()[0].split(':')
        pc_times.append(3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time)
        
    time_array = times.at[times.index[0],"pc4time"].split()[0].split(':')
    pc_times.append(3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]) - start_time)

    return [userID] + pc_times

In [11]:
compiled_pc_times_unordered = []
for i in range(num_subjs):
    compiled_pc_times_unordered.append(retreive_pc_times_unordered(i))
compiled_pc_times_unordered

[['j7WOU', 303.0, 357.0, 562.0, 612.0, 795.0, 849.0, 1025.0],
 ['GFPux', 262.0, 305.0, 490.0, 551.0, 764.0, 803.0, 1082.0],
 ['mOhA6', 326.0, 369.0, 549.0, 594.0, 779.0, 833.0, 1053.0],
 ['Ur76R', 279.0, 319.0, 500.0, 541.0, 705.0, 748.0, 948.0],
 ['2clJY', 338.0, 382.0, 560.0, 601.0, 802.0, 855.0, 1070.0],
 ['VRQJ7', 403.0, 454.0, 674.0, 721.0, 917.0, 964.0, 1167.0],
 ['5yMwL', 448.0, 499.0, 668.0, 709.0, 897.0, 935.0, 1133.0],
 ['mTe10', 345.0, 434.0, 649.0, 711.0, 918.0, 982.0, 1056.0],
 ['1XVI6', 282.0, 359.0, 576.0, 632.0, 896.0, 946.0, 1156.0],
 ['yFcTy', 265.0, 314.0, 521.0, 559.0, 743.0, 797.0, 988.0],
 ['zFHlS', 394.0, 443.0, 679.0, 730.0, 930.0, 974.0, 1174.0],
 ['Ssd0G', 244.0, 291.0, 493.0, 532.0, 699.0, 741.0, 924.0],
 ['ifh7Z', 361.0, 411.0, 584.0, 628.0, 812.0, 862.0, 1078.0],
 ['PPckh', 248.0, 300.0, 493.0, 547.0, 761.0, 807.0, 1004.0],
 ['yavQR', 465.0, 559.0, 794.0, 893.0, 1145.0, 1250.0, 1473.0],
 ['20EGr', 342.0, 398.0, 632.0, 695.0, 1015.0, 1057.0, 1299.0],
 ['RWgX

In [13]:
#create the summary stats output dataframe

temp_data = [['userID','pc1start','pc1end','pc2start','pc2end','pc3start','pc3end','pc4start']]
output_df = pd.DataFrame(temp_data, columns = ['userID','pc1start','pc1end','pc2start','pc2end','pc3start','pc3end','pc4start'])

k = 0
for i in range(num_subjs):
    dat = [retreive_pc_times_unordered(i)]
    temp_df = pd.DataFrame(dat, columns = ['userID','pc1start','pc1end','pc2start','pc2end','pc3start','pc3end','pc4start'])
    output_df.loc[k] = temp_df.loc[0]
    k += 1


In [14]:
output_df.to_csv(r'pc_times.csv', index = False)

In [ ]:
###  functions in reverse time index

In [50]:
#Get all of the video timestimes and put them in the same order
def retreive_vid_times(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    times = df_timestamps[df_timestamps["userID"] == userID]
    vid_order = times.at[times.index[0],'vidOrder'].split(',')
    time_array = times.at[times.index[0],"finalsurvey1time"].split()[0].split(':')
    end_time = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])
    
    vid_times = [[0,0],[0,0],[0,0],[0,0]]
    for i in range(len(vid_order)):
        if vid_order[i] == '0':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[0][0] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[0][1] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
        elif vid_order[i] == '1':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[1][0] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[1][1] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
        elif vid_order[i] == '2':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[2][0] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[2][1] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
        elif vid_order[i] == '3':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[3][0] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[3][1] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))

    return vid_times
    #!!! Units of time are "seconds from the end"

In [56]:
#compute summary statistics for each video for each participant
def compute_summary_stats(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    vid_times = compiled_vid_times[index]
    
    for r in rppg_data:
        if r[0] == userID:
            rppg_df = r[1]
            break
            
    disgust = []
    joy = []
    engagement = []
    valence = []
    rppg = []
    
    print(userID)  #progress tracker

    #compute the average affect in each facial affect category during the relevant times
    end_time = max(affect_df["time_stamp"])
    for i in range(4):
        #need to convert from "seconds from the end" to time_stamp
        time1 = end_time - vid_times[i][0]
        time2 = end_time - vid_times[i][1]
        rows = affect_df['time_stamp'].between(time1, time2)
        temp1 = []
        temp2 = []
        temp3 = []
        temp4 = []
        for j in rows.index:
            if rows[j]:
                temp1.append(affect_df.at[j,'disgust'])
                temp2.append(affect_df.at[j,'joy'])
                temp3.append(affect_df.at[j,'engagement'])
                temp4.append(affect_df.at[j,'valence'])
        disgust.append(stats.mean(temp1))
        joy.append(stats.mean(temp2))
        engagement.append(stats.mean(temp3))
        valence.append(stats.mean(temp4))
        
    #compute the average rPPG during the relevant times
    secs_per_bin = end_time / 240
    for i in range(4):
        #need to convert from "seconds from the end" to time_stamp
        #rPPG data is always split into 240 equal time bins
        time1 = 240 - (vid_times[i][0]/secs_per_bin)
        time2 = 240 - (vid_times[i][1]/secs_per_bin)
        rows = rppg_df['timebin'].between(time1, time2)
        temp1 = []
        for j in rows.index:
            if rows[j]:
                temp1.append(rppg_df.at[j,'hb-smooth'])
        rppg.append(stats.mean(temp1))

    #pull in the text sentiment polarity and subjectivity.  Values are -1 to 1, so *100 to compare with facial affect.
    sentiment = sentiment_df[sentiment_df["userID"] == userID]
    vid1_pol = sentiment.at[sentiment.index[0],'vid1_pol']*100
    vid2_pol = sentiment.at[sentiment.index[0],'vid2_pol']*100
    vid3_pol = sentiment.at[sentiment.index[0],'vid3_pol']*100
    vid4_pol = sentiment.at[sentiment.index[0],'vid4_pol']*100
    vid1_subj = sentiment.at[sentiment.index[0],'vid1_subj']*100
    vid2_subj = sentiment.at[sentiment.index[0],'vid2_subj']*100
    vid3_subj = sentiment.at[sentiment.index[0],'vid3_subj']*100
    vid4_subj = sentiment.at[sentiment.index[0],'vid4_subj']*100
    polarity_init = [vid1_pol,vid2_pol,vid3_pol,vid4_pol]
    subjectivity_init = [vid1_subj,vid2_subj,vid3_subj,vid4_subj]
    
    #reorder text sentiment based on video order to line up with the rest of the analysis
    times = df_timestamps[df_timestamps["userID"] == userID]
    vid_order = times.at[times.index[0],'vidOrder'].split(',')
    polarity = [0,0,0,0]
    subjectivity = [0,0,0,0]
    for i in range(len(vid_order)):
        if vid_order[i] == '0':
            polarity[0] = polarity_init[i]
            subjectivity[0] = subjectivity_init[i]
        elif vid_order[i] == '1':
            polarity[1] = polarity_init[i]
            subjectivity[1] = subjectivity_init[i]
        elif vid_order[i] == '2':
            polarity[2] = polarity_init[i]
            subjectivity[2] = subjectivity_init[i]
        elif vid_order[i] == '3':
            polarity[3] = polarity_init[i]
            subjectivity[3] = subjectivity_init[i]
        
    #Pull in user affiliation
    aff = retreive_affiliation(index)
    party = aff[1]
    ideology = aff[2]
    candidate = aff[3]
    like_joe = aff[4]
    like_trump = aff[5]
    
    #apply data to output variables based on user affiliation
    #currently we only accept data aligned (Dem-Lib-Biden or Rep-Con-Trump) so other cases are ignored
    if party == "Democrat":
        disgust_aligned = disgust[3]
        disgust_misaligned_proi = disgust[2]
        disgust_misaligned_prop = disgust[1]
        disgust_opposed = disgust[0]
        joy_aligned = joy[3]
        joy_misaligned_proi = joy[2]
        joy_misaligned_prop = joy[1]
        joy_opposed = joy[0]
        engagement_aligned = engagement[3]
        engagement_misaligned_proi = engagement[2]
        engagement_misaligned_prop = engagement[1]
        engagement_opposed = engagement[0]
        valence_aligned = valence[3]
        valence_misaligned_proi = valence[2]
        valence_misaligned_prop = valence[1]
        valence_opposed = valence[0]
        polarity_aligned = polarity[3]
        polarity_misaligned_proi = polarity[2]
        polarity_misaligned_prop = polarity[1]
        polarity_opposed = polarity[0]
        subjectivity_aligned = subjectivity[3]
        subjectivity_misaligned_proi = subjectivity[2]
        subjectivity_misaligned_prop = subjectivity[1]
        subjectivity_opposed = subjectivity[0]
        rppg_aligned = rppg[3]
        rppg_misaligned_proi = rppg[2]
        rppg_misaligned_prop = rppg[1]
        rppg_opposed = rppg[0]
    else:
        disgust_aligned = disgust[0]
        disgust_misaligned_proi = disgust[1]
        disgust_misaligned_prop = disgust[2]
        disgust_opposed = disgust[3]
        joy_aligned = joy[0]
        joy_misaligned_proi = joy[1]
        joy_misaligned_prop = joy[2]
        joy_opposed = joy[3]
        engagement_aligned = engagement[0]
        engagement_misaligned_proi = engagement[1]
        engagement_misaligned_prop = engagement[2]
        engagement_opposed = engagement[3]
        valence_aligned = valence[0]
        valence_misaligned_proi = valence[1]
        valence_misaligned_prop = valence[2]
        valence_opposed = valence[3]
        polarity_aligned = polarity[0]
        polarity_misaligned_proi = polarity[1]
        polarity_misaligned_prop = polarity[2]
        polarity_opposed = polarity[3]
        subjectivity_aligned = subjectivity[0]
        subjectivity_misaligned_proi = subjectivity[1]
        subjectivity_misaligned_prop = subjectivity[2]
        subjectivity_opposed = subjectivity[3]
        rppg_aligned = rppg[0]
        rppg_misaligned_proi = rppg[1]
        rppg_misaligned_prop = rppg[2]
        rppg_opposed = rppg[3]

    return [userID,party,ideology,candidate,like_joe,
              like_trump,disgust_aligned,disgust_misaligned_proi,
              disgust_misaligned_prop,disgust_opposed,joy_aligned,joy_misaligned_proi,
              joy_misaligned_prop,joy_opposed,engagement_aligned,engagement_misaligned_proi,
              engagement_misaligned_prop,engagement_opposed,valence_aligned,
              valence_misaligned_proi,valence_misaligned_prop,valence_opposed,
              polarity_aligned,polarity_misaligned_proi,polarity_misaligned_prop,
              polarity_opposed,subjectivity_aligned,subjectivity_misaligned_proi,
              subjectivity_misaligned_prop,subjectivity_opposed,rppg_aligned,
              rppg_misaligned_proi,rppg_misaligned_prop,rppg_opposed]